In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df = np.array(df)
m, n = df.shape
np.random.shuffle(df)

data_dev = df[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255

data_train = df[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255



In [5]:
def init_params():
    W1 = np.random.randn(10,784) * np.sqrt(1/784)
    b1 = np.zeros((10,1))
    W2 = np.random.randn(10,10) * np.sqrt(1/10)
    b2 = np.zeros((10,1))
    return W1,b1,W2,b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0))  # stability fix
    return expZ / np.sum(expZ, axis=0, keepdims=True)
    
def foward_prop(W1,b1,W2,b2,X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1,A1,Z2,A2

def one_hot(Y):
    one_hot_Y= np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0
    
def backwards_prop(A2, A1, Z1, Z2,W2, X,Y):
    one_hot_Y = one_hot(Y)
    m = X.shape[1]
    dZ2 = A2-one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = (1/m) * dZ1.dot(X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1-alpha*dW1
    b1 = b1-alpha*db1
    W2 = W2-alpha*dW2
    b2 = b2-alpha*db2
    return W1, b1, W2, b2
    
    
    

In [6]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y)/Y.size

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = foward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backwards_prop(A2, A1, Z1, Z2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            print("Accuracy: ", get_accuracy(get_predictions(A2),Y))
    return W1, b1, W2, b2

In [7]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 500, 0.10 )

Iteration:  0
[6 6 2 ... 2 3 6] [7 8 5 ... 0 2 5]
Accuracy:  0.12146341463414634
Iteration:  10
[1 3 2 ... 2 2 2] [7 8 5 ... 0 2 5]
Accuracy:  0.3352439024390244
Iteration:  20
[1 3 2 ... 0 2 3] [7 8 5 ... 0 2 5]
Accuracy:  0.48134146341463413
Iteration:  30
[7 8 0 ... 0 2 3] [7 8 5 ... 0 2 5]
Accuracy:  0.5638780487804878
Iteration:  40
[7 8 0 ... 0 2 3] [7 8 5 ... 0 2 5]
Accuracy:  0.6257317073170732
Iteration:  50
[7 8 0 ... 0 2 3] [7 8 5 ... 0 2 5]
Accuracy:  0.6939268292682926
Iteration:  60
[7 8 0 ... 0 2 3] [7 8 5 ... 0 2 5]
Accuracy:  0.7525121951219512
Iteration:  70
[7 8 0 ... 0 2 5] [7 8 5 ... 0 2 5]
Accuracy:  0.7784634146341464
Iteration:  80
[7 8 0 ... 0 2 5] [7 8 5 ... 0 2 5]
Accuracy:  0.7955121951219513
Iteration:  90
[7 8 0 ... 0 2 5] [7 8 5 ... 0 2 5]
Accuracy:  0.8088780487804879
Iteration:  100
[7 8 5 ... 0 2 5] [7 8 5 ... 0 2 5]
Accuracy:  0.8200975609756097
Iteration:  110
[7 8 5 ... 0 2 5] [7 8 5 ... 0 2 5]
Accuracy:  0.8304146341463414
Iteration:  120
[7 8 5 ..